# Provisioning OpenRouter API keys

This notebook is for instructors to provision OpenRouter API keys for workshop participants. It is not intended for use by participants.

However participants are welcome to see the code and use it as a reference whenever they need to bulk provision API keys for their own projects.

## OpenRouter API provisioning

OpenRouter provides a simple API to create and manage API keys. 

The core idea is to use a special API key that has the `admin` role, which allows you to create new API keys for other users. This key does not make regular API calls but is used to manage API keys.

This type of key is extremely powerful and should be kept secret. It is not intended for use in production applications or by end users.

If this type of key is compromised, it can be used to create new API keys for any user, which can then be used to make API calls and drain your account's balance to zero.

In [4]:
# first let's load needed libraries and also read system environment variables
# let's load Python sys to see what Python version we are using
import sys
print(f"Python version: {sys.version}")
# now datetime
from datetime import datetime
print(f"Current date and time: {datetime.now()}")
# we will need json to read and write JSON files
import json
# and os to read environment variables
import os

# now let's read environment variables

# OPEN_ROUTER_BSSDH_PROVISIONER
open_router_bssdh_provisioner = os.getenv("OPEN_ROUTER_BSSDH_PROVISIONER")
if open_router_bssdh_provisioner:
    print("OPEN_ROUTER_BSSDH_PROVISIONER key loaded from environment variables.")
    # README! we do not want to print the key itself for security reasons it is very easy to print it and publish it by mistake
else:
    print("OPEN_ROUTER_BSSDH_PROVISIONER key not found in environment variables. Please set it before running the script.")


# we will need some external libraries, so let's check if they are installed
# first requests
try:
    import requests
    print(f"requests library is installed: {requests.__version__}")
except ImportError:
    print("requests library is not installed. Please install it using 'pip install requests' from command line terminal.")

Python version: 3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]
Current date and time: 2025-07-25 12:06:09.517035
OPEN_ROUTER_BSSDH_PROVISIONER key loaded from environment variables.
requests library is not installed. Please install it using 'pip install requests' from command line terminal.
